In [72]:
import pandas as pd

threshold_pct_features = 0.50
threshold_pct_langauges = 1.0


# Analysis
### Features
Idea is to do some statistics on <i>feature</i> level to identify the fetures with the highest number of not-nan values.<br>
Please note that this is the first step in the analysis and followed by a similar analysis on <i>language</i> level.

In [73]:
#read denormalized feature sets.
df_denorm_features = pd.read_csv('data/vals_transformed.csv', sep=',', index_col = 'LANG_ID')

In [74]:
#features only; 
df_features = (df_denorm_features.reset_index(drop = True)  # 1. remove index col w/ LANG_UD
                                  .iloc[:, :-7]) #remove last n rows that contains language metadata

# df_features

In [75]:
num_languages = df_features.shape[0] #number of languages per feature (in case of European languages 62)

df_feat_nan_dist = df_features.isna().sum().to_frame() #calculate statistics about nan values per feature

df_feat_nan_dist = df_feat_nan_dist.rename(columns={0: 'cnt_not_given'})
df_feat_nan_dist['cnt_given'] = num_languages - df_feat_nan_dist['cnt_not_given'] #cnt for non nan values is (number of all languages / feature minus number of nan values)
df_feat_nan_dist['pct'] = df_feat_nan_dist['cnt_given'] / num_languages #relative values

df_feat_nan_dist = df_feat_nan_dist[(df_feat_nan_dist['pct'] >= threshold_pct_features)] #filter all features with less than 50% coverage

df_result_feature_analysis = df_feat_nan_dist.sort_values(by=['pct'], ascending=False) #order for convinience

# df_result_feature_analysis
df_result_feature_analysis.head(50)

,cnt_not_given,cnt_given,pct
53A,16,46,0.741935
97A,25,37,0.596774
83A,25,37,0.596774
82A,25,37,0.596774
81A,25,37,0.596774
87A,25,37,0.596774
95A,25,37,0.596774
143G,26,36,0.580645
143F,26,36,0.580645
138A,26,36,0.580645


### Language
Same idea: check number of non nan values per language and fetch top n percent

In [76]:
#List of features which have non nan values in more than 50% of languages in scope
lst = [i for i in df_result_feature_analysis.index] 

df_vals = df_denorm_features[lst]
df_vals

,53A,97A,83A,82A,81A,87A,95A,143G,143F,138A,...,51A,26A,72A,37A,92A,33A,101A,38A,90C,94A
LANG_ID,,,,,,,,,,,,,,,,,,,,,
alb,6.0,4.0,2.0,1.0,2.0,2.0,4.0,4.0,4.0,1.0,...,1.0,2.0,1.0,3.0,1.0,2.0,2.0,1.0,1.0,1.0
alt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
blr,7.0,5.0,3.0,3.0,7.0,1.0,5.0,4.0,4.0,1.0,...,1.0,2.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
bos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
urm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vep,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vot,7.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
num_features = df_vals.shape[1]

df_lang_nan_dist = df_vals.isnull().sum(axis=1).to_frame() #statistics about non nan values 
df_lang_nan_dist = df_lang_nan_dist.rename(columns={0: 'cnt_not_given'}) #rename column
df_lang_nan_dist['cnt_given'] = num_features - df_lang_nan_dist['cnt_not_given'] #calculate number of given features
df_lang_nan_dist['pct'] = df_lang_nan_dist['cnt_given'] / num_features #calculate ratio of non-nan features to all features

df_lang_nan_dist = df_lang_nan_dist.sort_values(by=['pct'], ascending=False) #sort by percentage descending

#filter all languages with coverage over 90
df_lang_nan_dist = df_lang_nan_dist[(df_lang_nan_dist['pct'] >= threshold_pct_langauges)]

df_lang_nan_dist.head(10)


,cnt_not_given,cnt_given,pct
LANG_ID,,,
rus,0,37,1.0
swe,0,37,1.0
iri,0,37,1.0
fre,0,37,1.0
fin,0,37,1.0
ita,0,37,1.0
est,0,37,1.0
eng,0,37,1.0
dut,0,37,1.0


# Create set of training data

In [78]:
df_training_set = df_denorm_features

# df_training_set
df_training_set = (df_training_set.filter(items=df_lang_nan_dist.index, axis=0) # filter languages with results from above
                                    .filter(items=df_result_feature_analysis.index, axis = 1)) #filter features with results from above

df_training_set.to_csv('data/training_set.csv', sep=',')
df_training_set.head(20)

,53A,97A,83A,82A,81A,87A,95A,143G,143F,138A,...,51A,26A,72A,37A,92A,33A,101A,38A,90C,94A
LANG_ID,,,,,,,,,,,,,,,,,,,,,
rus,7.0,3.0,2.0,1.0,2.0,1.0,4.0,4.0,4.0,1.0,...,1.0,2.0,4.0,5.0,3.0,2.0,1.0,5.0,1.0,1.0
swe,7.0,3.0,2.0,1.0,2.0,1.0,4.0,4.0,1.0,2.0,...,9.0,2.0,4.0,3.0,6.0,2.0,1.0,2.0,1.0,1.0
iri,7.0,4.0,2.0,2.0,3.0,2.0,4.0,4.0,4.0,2.0,...,5.0,4.0,1.0,1.0,1.0,6.0,6.0,4.0,1.0,1.0
fre,7.0,4.0,2.0,1.0,2.0,2.0,4.0,4.0,1.0,2.0,...,7.0,2.0,4.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0
fin,7.0,3.0,2.0,1.0,2.0,1.0,3.0,4.0,4.0,2.0,...,1.0,2.0,1.0,5.0,3.0,2.0,6.0,5.0,1.0,1.0
ita,7.0,4.0,2.0,3.0,2.0,2.0,4.0,4.0,4.0,2.0,...,9.0,2.0,4.0,1.0,6.0,2.0,2.0,2.0,1.0,1.0
est,7.0,3.0,2.0,1.0,2.0,1.0,3.0,4.0,4.0,2.0,...,1.0,2.0,1.0,5.0,5.0,2.0,2.0,5.0,1.0,1.0
eng,7.0,3.0,2.0,1.0,2.0,1.0,4.0,4.0,4.0,2.0,...,9.0,2.0,4.0,1.0,6.0,2.0,1.0,1.0,1.0,1.0
dut,6.0,5.0,3.0,1.0,7.0,1.0,5.0,4.0,1.0,2.0,...,9.0,2.0,4.0,1.0,6.0,2.0,1.0,1.0,1.0,1.0
